In [6]:
from langchain_ollama import ChatOllama

# Initialize Ollama with a deterministic response
llm = ChatOllama(
    model="llama3.2:latest",
    temperature=0
)


In [7]:
from langchain_core.messages import AIMessage
from langchain_core.runnables import RunnableLambda

# Define a simple case-inverting parser
def parse(ai_message: AIMessage) -> str:
    """Parse the AI message by inverting case."""
    return ai_message.content.swapcase()

# Wrap the function in RunnableLambda
parse = RunnableLambda(parse)

# Chain model with parser
chain = llm | parse

# Invoke the model
response = chain.invoke("Hello, how are you?")
print(response)


i'M JUST A LANGUAGE MODEL, SO i DON'T HAVE EMOTIONS OR FEELINGS LIKE HUMANS DO. hOWEVER, i'M FUNCTIONING PROPERLY AND READY TO ASSIST YOU WITH ANY QUESTIONS OR TASKS YOU MAY HAVE! hOW CAN i HELP YOU TODAY?


In [9]:
from typing import Iterable
from langchain_core.messages import AIMessageChunk
from langchain_core.runnables import RunnableGenerator

# Define a streaming parser
def streaming_parse(chunks: Iterable[AIMessageChunk]) -> Iterable[str]:
    """Parse incoming message chunks and invert case."""
    for chunk in chunks:
        yield chunk.content.swapcase()

# Wrap it in RunnableGenerator
streaming_parse = RunnableGenerator(streaming_parse)

# Chain with Ollama
chain = llm | streaming_parse

# Stream the output
for chunk in chain.stream("Tell me a fun fact"):
    print(chunk, end=" | ", flush=True)


hERE | 'S |  ONE | :

 | dID |  YOU |  KNOW |  THAT |  HONEY |  NEVER |  SPO | ILS | ? |  aRCHAE | OLOGISTS |  HAVE |  FOUND |  POTS |  OF |  HONEY |  IN |  ANCIENT |  eGYPTIAN |  TOM | BS |  THAT |  ARE |  OVER |   | 3 | , | 000 |  YEARS |  OLD |  AND |  STILL |  PERFECTLY |  EDIBLE | ! |  hONEY | 'S |  UNIQUE |  COMPOSITION | , |  WITH |  ITS |  LOW |  MOISTURE |  CONTENT |  AND |  ACIDIC |  Ph | , |  MAKES |  IT |  AN |  IDEAL |  PRES | ERVATIVE | . |  iT | 'S |  ALSO |  A |  NATURAL |  ANTIB | ACTERIAL |  AGENT | , |  WHICH |  HELPS |  TO |  PREVENT |  THE |  GROWTH |  OF |  BACTERIA |  AND |  OTHER |  MICRO | ORGANISMS |  THAT |  CAN |  CAUSE |  SPOIL | AGE | . |  sO | , |  IF |  YOU |  EVER |  FIND |  YOURSELF |  IN |  ANCIENT |  eGYPT | , |  JUST |  GRAB |  A |  JAR |  OF |  HONEY |  FROM |  THE |  TOMB |  – |  IT | 'LL |  STILL |  BE |  SWEET | ! |  | 

In [11]:
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseOutputParser

# Define a boolean parser
class BooleanOutputParser(BaseOutputParser[bool]):
    """Custom parser that converts YES/NO to Boolean."""
    
    true_val: str = "YES"
    false_val: str = "NO"

    def parse(self, text: str) -> bool:
        """Parse the text output into a boolean."""
        cleaned_text = text.strip().upper()
        if cleaned_text not in (self.true_val.upper(), self.false_val.upper()):
            raise OutputParserException(
                f"Expected output to be {self.true_val} or {self.false_val}, but got '{cleaned_text}'."
            )
        return cleaned_text == self.true_val.upper()

    @property
    def _type(self) -> str:
        return "boolean_output_parser"

# Create parser instance
parser = BooleanOutputParser()

# Test the parser
print(parser.invoke("YES"))  # Output: True
print(parser.invoke("NO"))   # Output: False

# Trigger exception
try:
    print(parser.invoke("MAYBETEST"))
except Exception as e:
    print(f"Error: {e}")


True
False
Error: Expected output to be YES or NO, but got 'MAYBETEST'.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE 


In [12]:
from langchain_core.output_parsers import BaseGenerationOutputParser
from langchain_core.outputs import ChatGeneration, Generation
from typing import List

# Define an advanced parser
class StrInvertCase(BaseGenerationOutputParser[str]):
    """Parser that inverts case in responses."""
    
    def parse_result(self, result: List[Generation], *, partial: bool = False) -> str:
        """Parse model output and swap case."""
        if len(result) != 1:
            raise NotImplementedError("This parser only works with single-generation outputs.")
        
        generation = result[0]
        
        if not isinstance(generation, ChatGeneration):
            raise OutputParserException("This parser only works with chat generation.")

        return generation.message.content.swapcase()

# Chain with Ollama
chain = llm | StrInvertCase()

# Invoke the model
response = chain.invoke("Tell me a short sentence about yourself.")
print(response)


i'M AN ARTIFICIAL INTELLIGENCE DESIGNED TO PROVIDE INFORMATION AND ASSIST WITH TASKS, BUT i DON'T HAVE PERSONAL EXPERIENCES OR EMOTIONS LIKE HUMANS DO.
